上一个文件我是按照思路把train,test和sessions数据集中的数据过了一遍，到了进行特征工程的时候首先要强调把数据转换成适合机器学习的模型格式，减小模型搭建的难度，能够输出质量更高的结果，涉及的所有数据的特征都需要尽量去深挖出来，比如2个datetiem格式的数据，我们要考虑通过日期格式来获取年,月,日的数据，又考虑到这是一个订房日期的数据集，所以，节假日和季节的数据我们也争取能够在搭建工程时候展现出来。以此为例

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn as sk
import datetime
from datetime import date

import seaborn as sns
from sklearn.preprocessing import *
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
import seaborn as sns
%matplotlib inline

## Sessions 文件  
对于sessions文件，代码使用Sandro的特征工程Notebook（https://www.kaggle.com/svpons/feature-engineering)  按照id分组，把每个用户的action等类型的特征归成一条数据，特征显示了用户具体操作的次数等特征，代码很精彩，值得推敲和学习！

In [3]:
df_sessions = pd.read_csv('sessions_3_1.csv')

In [4]:
df_sessions.drop(columns=['Unnamed: 0'] ,inplace = True )

In [5]:
df_sessions['id'] = df_sessions['user_id']
df_sessions = df_sessions.drop(['user_id'],axis=1)

In [6]:
df_sessions.head(5)

,action,action_type,action_detail,device_type,secs_elapsed,id
0,lookup,NAN,NAN,Windows Desktop,319.0,d1mm9tcy42
1,search_results,click,view_search_results,Windows Desktop,67753.0,d1mm9tcy42
2,lookup,NAN,NAN,Windows Desktop,301.0,d1mm9tcy42
3,search_results,click,view_search_results,Windows Desktop,22141.0,d1mm9tcy42
4,lookup,NAN,NAN,Windows Desktop,957.0,jpmpj40cvk


In [7]:
f_act = df_sessions.action.value_counts().argsort()
f_act_detail = df_sessions.action_detail.value_counts().argsort()
f_act_type = df_sessions.action_type.value_counts().argsort()
f_dev_type = df_sessions.device_type.value_counts().argsort()

In [8]:
print(f_act.shape)
print(f_act_detail.shape)
print(f_act_type.shape)
print(f_dev_type.shape)

(117,)
(30,)
(9,)
(14,)


In [9]:
dgr_sess = df_sessions.groupby(['id'])

In [10]:
len(dgr_sess)

15588

In [11]:
df_sessions.shape

(1045778, 6)

In [12]:
f_act['header_userpic']

107

In [13]:
df_sessions.action.values

array(['lookup', 'search_results', 'lookup', ..., 'personalize',
       'similar_listings', 'index'], dtype=object)

In [14]:
#Loop on dgr_sess to create all the features.
samples = []
cont = 0
ln = len(dgr_sess)

for g in dgr_sess:
    if cont%10000 == 0:
        print("%s from %s" %(cont, ln))
    gr = g[1]   #data frame that comtains all the data for a groupby value 'zzywmcn0jv'
    
    l = []
    
    #the id    for example:'zzywmcn0jv'
    l.append(g[0])
    
    # number of total actions  共有多少条数据个数
    l.append(len(gr))
    
    sev = gr.secs_elapsed.fillna(0).values   #These values are used later.
    
    #action features
    #(how many times each value occurs, numb of unique values, mean and std)
    c_act = [0] * len(f_act)
    for i,v in enumerate(gr.action.values):
        c_act[f_act[v]] += 1
    _, c_act_uqc = np.unique(gr.action.values, return_counts=True)
    c_act += [len(c_act_uqc), np.mean(c_act_uqc), np.std(c_act_uqc)]
    l = l + c_act
    
    #action_detail features
    #(how many times each value occurs, numb of unique values, mean and std)
    c_act_detail = [0] * len(f_act_detail)
    for i,v in enumerate(gr.action_detail.values):
        c_act_detail[f_act_detail[v]] += 1 
    _, c_act_det_uqc = np.unique(gr.action_detail.values, return_counts=True)
    c_act_detail += [len(c_act_det_uqc), np.mean(c_act_det_uqc), np.std(c_act_det_uqc)]
    l = l + c_act_detail
    
    #action_type features
    #(how many times each value occurs, numb of unique values, mean and std
    #+ log of the sum of secs_elapsed for each value)
    l_act_type = [0] * len(f_act_type)
    c_act_type = [0] * len(f_act_type)
    for i,v in enumerate(gr.action_type.values):
        l_act_type[f_act_type[v]] += sev[i]   
        c_act_type[f_act_type[v]] += 1  
    l_act_type = np.log(1 + np.array(l_act_type)).tolist()
    _, c_act_type_uqc = np.unique(gr.action_type.values, return_counts=True)
    c_act_type += [len(c_act_type_uqc), np.mean(c_act_type_uqc), np.std(c_act_type_uqc)]
    l = l + c_act_type + l_act_type    
    
    #device_type features
    #(how many times each value occurs, numb of unique values, mean and std)
    c_dev_type  = [0] * len(f_dev_type)
    for i,v in enumerate(gr.device_type .values):
        c_dev_type[f_dev_type[v]] += 1 
    c_dev_type.append(len(np.unique(gr.device_type.values)))
    _, c_dev_type_uqc = np.unique(gr.device_type.values, return_counts=True)
    c_dev_type += [len(c_dev_type_uqc), np.mean(c_dev_type_uqc), np.std(c_dev_type_uqc)]        
    l = l + c_dev_type    
    
    #secs_elapsed features        
    l_secs = [0] * 5 
    l_log = [0] * 15
    if len(sev) > 0:
        #Simple statistics about the secs_elapsed values.
        l_secs[0] = np.log(1 + np.sum(sev))
        l_secs[1] = np.log(1 + np.mean(sev)) 
        l_secs[2] = np.log(1 + np.std(sev))
        l_secs[3] = np.log(1 + np.median(sev))
        l_secs[4] = l_secs[0] / float(l[1])
        
        #Values are grouped in 15 intervals. Compute the number of values
        #in each interval.
        log_sev = np.log(1 + sev).astype(int)
        l_log = np.bincount(log_sev, minlength=15).tolist()                      
    l = l + l_secs + l_log
    
    #The list l has the feature values of one sample.
    samples.append(l)
    cont += 1

0 from 15588
10000 from 15588


In [15]:
len(samples)

15588

In [16]:
#preparing objects    
samples = np.array(samples)
samp_ar = samples[:, 1:].astype(np.float16)
samp_id = samples[:, 0]   #The first element in obs is the id of the sample.

In [17]:
#Creating a dataframe with the computed features    
col_names = []    #name of the columns
for i in range(len(samples[0])-1):
    col_names.append('c_' + str(i)) 

In [18]:
#creating the dataframe        
df_agg_sess = pd.DataFrame(samp_ar, columns=col_names)
df_agg_sess['id'] = samp_id
df_agg_sess.index = df_agg_sess.id

In [19]:
df_agg_sess.shape

(15588, 214)

In [20]:
df_agg_sess.head(10)

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,...,c_204,c_205,c_206,c_207,c_208,c_209,c_210,c_211,c_212,id
id,,,,,,,,,,,,,,,,,,,,,
007gj7kqdk,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,007gj7kqdk
009a40t3dk,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4.0,4.0,5.0,2.0,1.0,1.0,2.0,0.0,009a40t3dk
00allnceb8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00allnceb8
00e8bokexa,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00e8bokexa
00fhpdik5t,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,2.0,10.0,4.0,3.0,0.0,0.0,0.0,0.0,00fhpdik5t
00fn6wu77e,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,4.0,5.0,5.0,3.0,1.0,0.0,2.0,0.0,00fn6wu77e
00fyswof3k,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,4.0,4.0,11.0,16.0,8.0,3.0,1.0,0.0,00fyswof3k
00guzlz8b8,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,8.0,2.0,4.0,4.0,6.0,4.0,1.0,0.0,00guzlz8b8
00iapy7gb3,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,2.0,2.0,1.0,1.0,0.0,1.0,0.0,00iapy7gb3


### Finish cleaning of sessions 

## 对train 和test做特征工程：

读取数据

In [21]:
train = pd.read_csv("train_3_1.csv")
test = pd.read_csv("test_3_1.csv")

In [22]:
print(train.shape,test.shape)
print(train.info(),test.info())

(213451, 18) (62096, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 18 columns):
Unnamed: 0                 213451 non-null int64
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     117763 non-null object
age                        122995 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
date_first_active

再次进行数据观察，我们发现我们之前对于一些数据格式的转换成datetime格式又变回来成了object了，所以我们还需要再操作一次。原本的index也存成了一列，我们将其和之前决定删除的date_first_booking这一列删去：

In [23]:
train.drop(columns= 'Unnamed: 0',inplace = True)

In [24]:
train.drop(columns= 'date_first_booking',inplace = True)

In [25]:
test.drop(columns= 'Unnamed: 0',inplace = True)

In [26]:
test.drop(columns= 'date_first_booking',inplace = True)

此外，我们把我们需要预测的数据(country_destination)保存到labels, id保存到 id_test里，这样我们submission的时候就可以用它们了。

In [27]:
train_row = train.shape[0]

labels = train['country_destination'].values

id_test = test['id']

train.drop(['country_destination'], axis = 1, inplace = True)

In [28]:
train.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,date_first_active
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2009-03-19
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2009-05-23
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2009-06-09
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2009-10-31
4,87mebub9p4,2010-09-14,20091208061105,NaN,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2009-12-08


In [29]:
test.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,date_first_active
0,5uwns89zht,2014-07-01,20140701000006,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,2014-07-01
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,2014-07-01
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,2014-07-01
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE,2014-07-01
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,2014-07-01


## 合并train 和 test

In [30]:
df = pd.concat([train, test], axis = 0, ignore_index = True)
df.shape

(275547, 15)

In [31]:
df.dtypes

id                          object
date_account_created        object
timestamp_first_active       int64
gender                      object
age                        float64
signup_method               object
signup_flow                  int64
language                    object
affiliate_channel           object
affiliate_provider          object
first_affiliate_tracked     object
signup_app                  object
first_device_type           object
first_browser               object
date_first_active           object
dtype: object

In [32]:
df.isnull().sum()

id                              0
date_account_created            0
timestamp_first_active          0
gender                     129480
age                        119645
signup_method                   0
signup_flow                     0
language                        0
affiliate_channel               0
affiliate_provider              0
first_affiliate_tracked      6085
signup_app                      0
first_device_type               0
first_browser                   0
date_first_active               0
dtype: int64

再次进行一个个的特征处理，与数据探索不同的是，在特征工程的时候我们要考虑2个方面：   
1.数据能够让机器更有效率更容易的识别   
2.尽量挖掘出更多的特征分类信息   

#### timestamp_first_active

In [33]:
tfa = pd.to_datetime(df.timestamp_first_active // 1000000, format = '%Y%m%d')

In [34]:
df.date_first_active.isnull().sum()

0

再次转变成datetime格式，然后查看没有空值，可以继续：

把年月日的数据分成3个特征：

In [35]:
df['tfa_year'] = np.array([x.year for x in tfa])
df['tfa_month'] = np.array([x.month for x in tfa])
df['tfa_day'] = np.array([x.day for x in tfa])

In [36]:
df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,date_first_active,tfa_year,tfa_month,tfa_day
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2009-03-19,2009,3,19
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,2009-05-23,2009,5,23
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,2009-06-09,2009,6,9
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,2009-10-31,2009,10,31
4,87mebub9p4,2010-09-14,20091208061105,NaN,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,2009-12-08,2009,12,8


通过isoweekday()函数创建星期天函数(利用离散编码  One hot）：

In [37]:
df['tfa_wd'] = np.array([x.isoweekday() for x in tfa])
df_tfa_wd = pd.get_dummies(df.tfa_wd, prefix = 'tfa_wd')
df = pd.concat((df, df_tfa_wd), axis = 1)
df.drop(['tfa_wd'], axis = 1, inplace = True)
df.head(3)

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,NaN,facebook,0,en,direct,direct,...,2009,3,19,0,0,0,1,0,0,0
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,2009,5,23,0,0,0,0,0,1,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,2009,6,9,0,1,0,0,0,0,0


这里有一个难点，订房季节的特征也很重要，我们定义一个季节的日期周期，由于我们数据年份跨度很大，这里会出现闰年的情况，设置周期的时候最好用一个闰年的年份去试一下：

In [38]:
Y = 2000 
seasons = [(0, (date(Y,  1,  1),  date(Y,  3, 20))),  #开头冬天
           (1, (date(Y,  3, 21),  date(Y,  6, 20))),  #春天
           (2, (date(Y,  6, 21),  date(Y,  9, 22))),  #夏天
           (3, (date(Y,  9, 23),  date(Y, 12, 20))),  #秋天
           (0, (date(Y, 12, 21),  date(Y, 12, 31)))]  #末尾冬天

创建自定义函数来区分季节，仍旧用one hot 编码创建新的季节特征：

In [39]:
def get_season(dt):
    dt = dt.date()
    dt = dt.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= dt <= end)


In [40]:
df['tfa_season'] = np.array([get_season(x) for x in tfa])
df_tfa_season = pd.get_dummies(df.tfa_season, prefix = 'tfa_season')
df = pd.concat((df, df_tfa_season), axis = 1)
df.drop(['tfa_season'], axis = 1, inplace = True)
df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,tfa_season_0,tfa_season_1,tfa_season_2,tfa_season_3
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,NaN,facebook,0,en,direct,direct,...,0,0,1,0,0,0,1,0,0,0
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,0,0,0,0,1,0,0,1,0,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,1,0,0,0,0,0,0,1,0,0
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,...,0,0,0,0,1,0,0,0,0,1
4,87mebub9p4,2010-09-14,20091208061105,NaN,41.0,basic,0,en,direct,direct,...,1,0,0,0,0,0,0,0,0,1


#### date_account_created

对于账号生成时间，方法和timestamp_first_active账号激活时间我们采取一样的特征处理，但是我们会猜想一点！账号生成时间和账号激活时间的间隔是不是也能作为一个特征来帮助我们建立更好的模型，我们需要它：

同样的处理方法：

In [41]:
dac = pd.to_datetime(df.date_account_created)

In [42]:
df['dac_year'] = np.array([x.year for x in dac])
df['dac_month'] = np.array([x.month for x in dac])
df['dac_day'] = np.array([x.day for x in dac])

In [43]:
df['dac_wd'] = np.array([x.isoweekday() for x in dac])
df_dac_wd = pd.get_dummies(df.dac_wd, prefix = 'dac_wd')
df = pd.concat((df, df_dac_wd), axis = 1)
df.drop(['dac_wd'], axis = 1, inplace = True)

In [44]:
df['dac_season'] = np.array([get_season(x) for x in dac])
df_dac_season = pd.get_dummies(df.dac_season, prefix = 'dac_season')
df = pd.concat((df, df_dac_season), axis = 1)
df.drop(['dac_season'], axis = 1, inplace = True)
df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,dac_wd_2,dac_wd_3,dac_wd_4,dac_wd_5,dac_wd_6,dac_wd_7,dac_season_0,dac_season_1,dac_season_2,dac_season_3
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,NaN,facebook,0,en,direct,direct,...,0,0,0,0,0,0,0,0,1,0
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,0,1,0,0,0,0,0,1,0,0
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,1,0,0,0,0,0,0,0,0,1
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,...,0,0,0,0,0,0,0,0,0,1
4,87mebub9p4,2010-09-14,20091208061105,NaN,41.0,basic,0,en,direct,direct,...,1,0,0,0,0,0,0,0,1,0


这里我们用tfc日期减去dac日期就能得到一个间隔天数：

In [45]:
dt_span = dac.subtract(tfa).dt.days

In [46]:
dt_span.describe()

count    275547.000000
mean          0.179577
std          10.516688
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        1456.000000
dtype: float64

In [47]:
dt_span.unique()

array([ 466,  732,  476,  765,  280,    0,  312,  101,    7,  262,  356,
        216, 1369,    6,  163, 1456,  665,  308, 1361,  354,  274,  218,
         76,  349,  851,  995,    2,  998,  154, 1036,  616,  722,  516,
       1011, 1148,   20,  768,   91,  448,   13,    4,  337,  399,  710,
       1018,  328,    1,    3,  311,  241,   78,  718,  304,  259,  303,
         62,  658,  838,  167,  507,  239,  401,  283,  882,  636,   37,
        451,   16,  565,  634,  358,  137,  314,   95,  774,  169,   31,
        204,  763,  587,   40,   57,  125,  124,   47,  176,  659,  595,
        514,   18,   44,   86,  431,  129,  322,  622,  234,   80,    5,
        138,   65,   29,  395,   92,  153,   11,  438,  103,  367,  369,
        447,  364,  133,  210,  362,  237,  118,  249,   17,  143,  116,
        108,  229,  331,   74,  150,  146,  359,  189,  258,    9,  104,
         46,  179,   10,  132,  246,  243,   35,   21,  106,  111,  144],
      dtype=int64)

可看出大多数是账号生成完就激活了的，符合客户操作的逻辑。可是仍然有客户注册和激活时间差了1456天，这太极端了，我们按这可以把这个间隔天数化为一个新特征：间隔一天以下的，间隔小于一个月的，间隔小于一年的，和间隔一年以上的。

In [48]:
def get_span(dt):
    # dt is an integer
    if dt == 0:
        return 'OneDay'
    elif (dt < 31) & (dt > 0):
        return 'OneMonth'
    elif (dt >= 31) & (dt <= 365):
        return 'OneYear'
    else:
        return 'other'

In [49]:
df['dt_span'] = np.array([get_span(x) for x in dt_span])
df_dt_span = pd.get_dummies(df.dt_span, prefix = 'dt_span')
df = pd.concat((df, df_dt_span), axis = 1)
df.drop(['dt_span'], axis = 1, inplace = True)
df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,dac_wd_6,dac_wd_7,dac_season_0,dac_season_1,dac_season_2,dac_season_3,dt_span_OneDay,dt_span_OneMonth,dt_span_OneYear,dt_span_other
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,NaN,facebook,0,en,direct,direct,...,0,0,0,0,1,0,0,0,0,1
1,820tgsjxq7,2011-05-25,20090523174809,MALE,38.0,facebook,0,en,seo,google,...,0,0,0,1,0,0,0,0,0,1
2,4ft3gnwmtx,2010-09-28,20090609231247,FEMALE,56.0,basic,3,en,direct,direct,...,0,0,0,0,0,1,0,0,0,1
3,bjjt8pjhuk,2011-12-05,20091031060129,FEMALE,42.0,facebook,0,en,direct,direct,...,0,0,0,0,0,1,0,0,0,1
4,87mebub9p4,2010-09-14,20091208061105,NaN,41.0,basic,0,en,direct,direct,...,0,0,0,0,1,0,0,0,1,0


在特征工程完这个特征后，我们把原本的数据丢掉：

In [50]:
df.drop(['date_account_created','timestamp_first_active','date_first_active'], axis = 1, inplace = True)

#### Age

我们看一下我们之前处理的数据：    
1.把高于2009的（大多数是注册年份2014）转换为空值了   
2.我们观察了数据之后又把在2000岁左右的异常值用2012减去了(设想为用户填错填成出生年份）   
3.接着把13岁以下和95岁以上的人群转换为空值了   

In [51]:
df.age.describe()

count    155902.000000
mean         36.030449
std          11.588463
min          13.000000
25%          28.000000
50%          33.000000
75%          42.000000
max          95.000000
Name: age, dtype: float64

In [52]:
df.age.value_counts()

30.0    7712
28.0    7687
29.0    7647
31.0    7540
27.0    7462
        ... 
89.0      18
91.0      16
15.0       9
13.0       5
14.0       1
Name: age, Length: 83, dtype: int64

在年龄特征中，有一项特征我们要区分开来，就是年龄段的划分，不同年龄段中有着不同的选择目标这是正常的，所以我们需要通过年龄段增加一个分类特征，并且我们让age里面的空值自行为一个特征元素：

In [53]:
age = df.age
age.fillna(-1,inplace = True)

自定义一个函数用于建立年龄段，我们按照15岁为一个年龄段进行划分(小于15岁的定义很年轻，大于90岁的定义很老）：

In [54]:
divide = 15
def get_age(age):
    if age < 0 :
        return 'Nan'
    elif age <= divide :
        return divide
    elif (age <= divide * 2):
        return divide * 2
    elif (age <= divide * 3):
        return divide * 3
    elif (age <= divide * 4):
        return divide * 4
    elif (age <= divide * 5):
        return divide * 5
    elif (age <= divide * 6):
        return divide * 6
    else:
        return 'Super_old'

In [55]:
df['age'] = np.array([get_age(x) for x in age])
df_age = pd.get_dummies(df.age, prefix = 'age')
df = pd.concat((df, df_age), axis = 1)
df.drop(['age'], axis = 1, inplace = True)
df.head()

,id,gender,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,...,dt_span_OneYear,dt_span_other,age_15,age_30,age_45,age_60,age_75,age_90,age_Nan,age_Super_old
0,gxn3p5htnn,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,...,0,1,0,0,0,0,0,0,1,0
1,820tgsjxq7,MALE,facebook,0,en,seo,google,untracked,Web,Mac Desktop,...,0,1,0,0,1,0,0,0,0,0
2,4ft3gnwmtx,FEMALE,basic,3,en,direct,direct,untracked,Web,Windows Desktop,...,0,1,0,0,0,1,0,0,0,0
3,bjjt8pjhuk,FEMALE,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,...,0,1,0,0,1,0,0,0,0,0
4,87mebub9p4,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,...,1,0,0,0,1,0,0,0,0,0


### Gender

In [56]:
df.gender.isnull().value_counts()

False    146067
True     129480
Name: gender, dtype: int64

记得性别里面我们把unknown都换成了空值，我们可以对空值进行转换成-1,来创建特征：

In [57]:
df.gender.fillna(-1,inplace = True)

In [58]:
df['fc_gender'] = np.array([x for x in df.gender])
df_fc_gender = pd.get_dummies(df.fc_gender, prefix = 'fc_gender')
df = pd.concat((df, df_fc_gender), axis = 1)
df.drop(['fc_gender'], axis = 1, inplace = True)
df.head()

,id,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,...,age_60,age_75,age_90,age_Nan,age_Super_old,fc_gender,fc_gender_-1,fc_gender_FEMALE,fc_gender_MALE,fc_gender_OTHER
0,gxn3p5htnn,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,...,0,0,0,1,0,-1,1,0,0,0
1,820tgsjxq7,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,...,0,0,0,0,0,MALE,0,0,1,0
2,4ft3gnwmtx,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,...,1,0,0,0,0,FEMALE,0,1,0,0
3,bjjt8pjhuk,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,...,0,0,0,0,0,FEMALE,0,1,0,0
4,87mebub9p4,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,...,0,0,0,0,0,-1,1,0,0,0


### first_affiliate_tracked

这是最后一个有空值的特征，我们还是把空值换成-1来创建特征：

In [59]:
df.first_affiliate_tracked.isnull().sum()

6085

In [60]:
df.first_affiliate_tracked.fillna(-1,inplace = True)

In [61]:
df['fc_first_affiliate_tracked'] = np.array([x for x in df.first_affiliate_tracked])
df_fc_first_affiliate_tracked = pd.get_dummies(df.fc_first_affiliate_tracked, prefix = 'fc_fac')
df = pd.concat((df, df_fc_first_affiliate_tracked), axis = 1)
df.drop(['fc_first_affiliate_tracked'], axis = 1, inplace = True)
df.head()

,id,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,signup_app,first_device_type,first_browser,tfa_year,...,fc_gender_OTHER,fc_first_affiliate_tracked,fc_fac_-1,fc_fac_linked,fc_fac_local ops,fc_fac_marketing,fc_fac_omg,fc_fac_product,fc_fac_tracked-other,fc_fac_untracked
0,gxn3p5htnn,facebook,0,en,direct,direct,Web,Mac Desktop,Chrome,2009,...,0,untracked,0,0,0,0,0,0,0,1
1,820tgsjxq7,facebook,0,en,seo,google,Web,Mac Desktop,Chrome,2009,...,0,untracked,0,0,0,0,0,0,0,1
2,4ft3gnwmtx,basic,3,en,direct,direct,Web,Windows Desktop,IE,2009,...,0,untracked,0,0,0,0,0,0,0,1
3,bjjt8pjhuk,facebook,0,en,direct,direct,Web,Mac Desktop,Firefox,2009,...,0,untracked,0,0,0,0,0,0,0,1
4,87mebub9p4,basic,0,en,direct,direct,Web,Mac Desktop,Chrome,2009,...,0,untracked,0,0,0,0,0,0,0,1


 ### 剩下的特征也都是分类的问题，我们集中用one hot 编码来创建一下：

In [62]:
feat_toOHE = ['signup_method', 
             'signup_flow', 
             'language', 
             'affiliate_channel', 
             'affiliate_provider', 
             'signup_app', 
             'first_device_type', 
             'first_browser']

In [63]:
for f in feat_toOHE:
    df_ohe = pd.get_dummies(df[f], prefix=f, dummy_na=True)
    df.drop([f], axis = 1, inplace = True)
    df = pd.concat((df, df_ohe), axis = 1)

In [64]:
df.head()

,id,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,...,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser,first_browser_nan
0,gxn3p5htnn,2009,3,19,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,820tgsjxq7,2009,5,23,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,4ft3gnwmtx,2009,6,9,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,bjjt8pjhuk,2009,10,31,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,87mebub9p4,2009,12,8,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
df.shape

(275547, 205)

到这里我们的特征工程就创建完毕啦，一共有205个特征。


## 合并sessions 和 train,test数据集：

把之前我们用来groupby 的index 的名字丢掉：

In [66]:
df_agg_sess.index.name = None

In [67]:
df_all = pd.merge(df, df_agg_sess, how='left')

再把df数据集中在session的空值我们给予一个新的特征 -2

In [68]:
df_all
df_all = df_all.drop(['id'], axis=1)
df_all = df_all.fillna(-2)

In [80]:
df_all['all_null'] = np.array([sum(r<0) for r in df_all.values])

In [76]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [81]:
df_all.head()

,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,tfa_season_0,tfa_season_1,tfa_season_2,tfa_season_3,dac_year,dac_month,dac_day,dac_wd_1,dac_wd_2,dac_wd_3,dac_wd_4,dac_wd_5,dac_wd_6,dac_wd_7,dac_season_0,dac_season_1,dac_season_2,dac_season_3,dt_span_OneDay,dt_span_OneMonth,dt_span_OneYear,dt_span_other,age_15,age_30,age_45,age_60,age_75,age_90,age_Nan,age_Super_old,fc_gender_-1,fc_gender_FEMALE,fc_gender_MALE,fc_gender_OTHER,fc_fac_-1,fc_fac_linked,fc_fac_local ops,fc_fac_marketing,fc_fac_omg,fc_fac_product,fc_fac_tracked-other,fc_fac_untracked,signup_method_basic,signup_method_facebook,signup_method_google,signup_method_weibo,signup_method_nan,signup_flow_0.0,signup_flow_1.0,signup_flow_2.0,signup_flow_3.0,signup_flow_4.0,signup_flow_5.0,signup_flow_6.0,signup_flow_8.0,signup_flow_10.0,signup_flow_12.0,signup_flow_14.0,signup_flow_15.0,signup_flow_16.0,signup_flow_20.0,signup_flow_21.0,signup_flow_23.0,signup_flow_24.0,signup_flow_25.0,signup_flow_nan,language_-unknown-,language_ca,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,language_hr,language_hu,language_id,language_is,language_it,language_ja,language_ko,language_nl,language_no,language_pl,language_pt,language_ru,language_sv,language_th,language_tr,language_zh,language_nan,affiliate_channel_api,affiliate_channel_content,affiliate_channel_direct,affiliate_channel_other,affiliate_channel_remarketing,affiliate_channel_sem-brand,affiliate_channel_sem-non-brand,affiliate_channel_seo,affiliate_channel_nan,affiliate_provider_baidu,affiliate_provider_bing,affiliate_provider_craigslist,affiliate_provider_daum,affiliate_provider_direct,affiliate_provider_email-marketing,affiliate_provider_facebook,affiliate_provider_facebook-open-graph,affiliate_provider_google,affiliate_provider_gsp,affiliate_provider_meetup,affiliate_provider_naver,affiliate_provider_other,affiliate_provider_padmapper,affiliate_provider_vast,affiliate_provider_wayn,affiliate_provider_yahoo,affiliate_provider_yandex,affiliate_provider_nan,signup_app_Android,signup_app_Moweb,signup_app_Web,signup_app_iOS,signup_app_nan,first_device_type_Android Phone,first_device_type_Android Tablet,first_device_type_Desktop (Other),first_device_type_Mac Desktop,first_device_type_Other/Unknown,first_device_type_SmartPhone (Other),first_device_type_Windows Desktop,first_device_type_iPad,first_device_type_iPhone,first_device_type_nan,first_browser_-unknown-,first_browser_AOL Explorer,first_browser_Android Browser,first_browser_Apple Mail,first_browser_Arora,first_browser_Avant Browser,first_browser_BlackBerry Browser,first_browser_Camino,first_browser_Chrome,first_browser_Chrome Mobile,first_browser_Chromium,first_browser_CometBird,first_browser_Comodo Dragon,first_browser_Conkeror,first_browser_CoolNovo,first_browser_Crazy Browser,first_browser_Epic,first_browser_Firefox,first_browser_Flock,first_browser_Google Earth,first_browser_Googlebot,first_browser_IBrowse,first_browser_IE,first_browser_IE Mobile,first_browser_IceDragon,first_browser_IceWeasel,first_browser_Iron,first_browser_Kindle Browser,first_browser_Maxthon,first_browser_Mobile Firefox,first_browser_Mobile Safari,first_browser_Mozilla,first_browser_NetNewsWire,first_browser_Nintendo Browser,first_browser_OmniWeb,first_browser_Opera,first_browser_Opera Mini,first_browser_Opera Mobile,first_browser_Outlook 2007,first_browser_PS Vita browser,first_browser_Pale Moon,first_browser_Palm Pre web browser,first_browser_RockMelt,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser,first_browser_nan,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31

## 分离训练集和测试集：

我们之前定义了一个train.row，我们用它来split 训练集和测试集：

In [82]:
Finish_train = df_all.iloc[:train_row, :]
Finish_test = df_all.iloc[train_row:, :]

In [83]:
Finish_train.shape

(213451, 416)

In [85]:
Finish_test.shape

(62096, 416)

然后存挡下来：

In [88]:
Finish_train.to_csv("Finish_train.csv")
Finish_test.to_csv("Finish_test.csv")
labels.tofile("Airbnb_ytrain_v2.csv", sep='\n', format='%s')